In [1]:
import pandas as pd
import numpy as np

import xgboost as xgb
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import ElasticNetCV, RidgeCV
from sklearn.ensemble import RandomForestRegressor, StackingRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

from bayes_opt import BayesianOptimization

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasRegressor

In [2]:
seed = 0
np.random.seed(seed)
tf.random.set_seed(seed = seed)

# Reading and splitting the data

In [3]:
df = pd.read_csv("sf_sold_houses_clean.csv")

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1797 entries, 0 to 1796
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   bedrooms                1797 non-null   int64  
 1   bathrooms               1797 non-null   int64  
 2   build_year              1797 non-null   int64  
 3   build_type              1797 non-null   object 
 4   area                    1797 non-null   int64  
 5   lot_area                1797 non-null   int64  
 6   parking_spots           1797 non-null   int64  
 7   homeowners_association  1797 non-null   int64  
 8   zip_code                1797 non-null   object 
 9   nearby_elem_school      1797 non-null   float64
 10  nearby_middle_school    1797 non-null   float64
 11  nearby_high_school      1797 non-null   float64
 12  nh_median_price         1797 non-null   int64  
 13  nh_days_on_market       1797 non-null   int64  
 14  nh_price_per_sqft       1797 non-null   

In [5]:
X = df.drop(columns = ["price"])
y = df["price"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = seed)

In [6]:
X_train.shape, X_test.shape

((1437, 18), (360, 18))

# Elastic Net Regression

In [7]:
numerical_types = ["int64", "float64"]
numerical_features = X_train.select_dtypes(include = numerical_types).columns
numerical_transformer = StandardScaler()

categorical_types = ["object"]
categorical_features = X_train.select_dtypes(include = categorical_types).columns
categorical_transformer = OneHotEncoder(handle_unknown = "ignore")

preprocessor = ColumnTransformer(
    transformers = [
        ("numerical", numerical_transformer, numerical_features),
        ("categorical", categorical_transformer, categorical_features)
    ]
)

elastic_net = ElasticNetCV(l1_ratio = [0.01, 0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1], random_state = seed)

elastic_net_model = Pipeline(
    steps = [("preprocessing", preprocessor), ("elastic_net", elastic_net)]
)

res = elastic_net_model.fit(X_train, y_train)

In [8]:
print("Elastic Net parameters", 
      "\nalpha: {}".format(round(elastic_net_model["elastic_net"].alpha_, 2)),
      "\nl1 ratio: {}".format(round(elastic_net_model["elastic_net"].l1_ratio_, 2)))

Elastic Net parameters 
alpha: 1334.61 
l1 ratio: 1.0


In [9]:
elastic_net_pred = elastic_net_model.predict(X_test)
print("Elastic Net testing results", 
      "\nRMSE: {}".format(round(mean_squared_error(y_test, elastic_net_pred, squared = False), 4)), 
      "\nMAE: {}".format(round(mean_absolute_error(y_test, elastic_net_pred), 4)), 
      "\nMAPE: {}".format(round(mean_absolute_percentage_error(y_test, elastic_net_pred), 4)))

Elastic Net testing results 
RMSE: 392897.4235 
MAE: 282319.0366 
MAPE: 0.1981


# Random Forest

In [10]:
rf_pipe = Pipeline(
    steps = [("preprocessing", OneHotEncoder(handle_unknown = "ignore")), 
             ("rf", RandomForestRegressor(random_state = seed))]
)

rf_param_grid = {
    "rf__n_estimators": [100, 500, 1000], 
    "rf__max_features": [0.33, 0.67, 1.0]
}

random_forest_model = GridSearchCV(rf_pipe, rf_param_grid, n_jobs = -1)
res = random_forest_model.fit(X_train, y_train)

In [11]:
random_forest_model.best_params_

{'rf__max_features': 0.67, 'rf__n_estimators': 100}

In [12]:
rf_pred = random_forest_model.predict(X_test)
print("Random Forest testing results", 
      "\nRMSE: {}".format(round(mean_squared_error(y_test, rf_pred, squared = False), 4)), 
      "\nMAE: {}".format(round(mean_absolute_error(y_test, rf_pred), 4)), 
      "\nMAPE: {}".format(round(mean_absolute_percentage_error(y_test, rf_pred), 4)))

Random Forest testing results 
RMSE: 442219.1401 
MAE: 301930.2784 
MAPE: 0.2055


# XGBoost (grid search and Bayesian optimization)

## Tuning hyperparameters using grid search

In [13]:
xgb_pipe = Pipeline(
    steps = [("preprocessing", OneHotEncoder(handle_unknown = "ignore")), 
             ("xgb", xgb.XGBRegressor(random_state = seed))]
)

xgb_param_grid = {
    "xgb__n_estimators": [5, 10, 20, 50, 100, 200], 
    "xgb__max_depth": [3, 5, 7, 10],
    "xgb__gamma": [0, 0.01, 0.1, 0.5],
    "xgb__learning_rate": [0.01, 0.1, 0.2, 0.3]
}

xgb_model = GridSearchCV(xgb_pipe, xgb_param_grid, n_jobs = -1)
res = xgb_model.fit(X_train, y_train)

In [14]:
xgb_gs_params = xgb_model.best_params_
xgb_gs_params

{'xgb__gamma': 0,
 'xgb__learning_rate': 0.2,
 'xgb__max_depth': 3,
 'xgb__n_estimators': 200}

## Tuning hyperparameters using Bayesian optimization

In [15]:
xgb.set_config(verbosity = 0)
dtrain = xgb.DMatrix(OneHotEncoder(handle_unknown = "ignore").fit_transform(X_train), label = y_train)

def get_xgb_rmse(n_estimators, max_depth, gamma, learning_rate):
    params = {'max_depth': int(max_depth), 
              'gamma': gamma, 
              'n_estimators': int(n_estimators), 
              'learning_rate': learning_rate, 
              'eval_metric': 'rmse'}
    cv_result = xgb.cv(params, dtrain, num_boost_round = 70, nfold = 5)
    return -1.0 * cv_result['test-rmse-mean'].iloc[-1]

xgb_bo = BayesianOptimization(get_xgb_rmse, {'max_depth': (3, 10),
                                             'gamma': (0, 1),
                                             'learning_rate': (0, 1),
                                             'n_estimators': (10, 300)
                                            }
                             )

xgb_bo.maximize(n_iter = 50, init_points = 50, acq = 'ei', random_state = seed)

|   iter    |  target   |   gamma   | learni... | max_depth | n_esti... |
-------------------------------------------------------------------------
|  1        | -5.01e+05 |  0.9195   |  0.7133   |  5.336    |  250.5    |
|  2        | -4.978e+0 |  0.5245   |  0.6183   |  8.565    |  234.2    |
|  3        | -5.203e+0 |  0.07066  |  0.92     |  4.081    |  85.57    |
|  4        | -1.294e+0 |  0.9377   |  0.004175 |  7.185    |  108.6    |
|  5        | -5.27e+05 |  0.3972   |  0.8264   |  7.721    |  184.6    |
|  6        | -4.871e+0 |  0.04092  |  0.143    |  6.974    |  258.6    |
|  7        | -5.541e+0 |  0.9969   |  0.9759   |  7.801    |  78.53    |
|  8        | -5.013e+0 |  0.6139   |  0.6677   |  9.885    |  248.3    |
|  9        | -5.006e+0 |  0.5106   |  0.6468   |  6.017    |  172.9    |
|  10       | -5.321e+0 |  0.8868   |  0.9595   |  6.993    |  263.5    |
|  11       | -4.928e+0 |  0.5516   |  0.1057   |  7.345    |  169.3    |
|  12       | -6.115e+0 |  0.6169   | 

In [16]:
xgb_bo_params = xgb_bo.max["params"]
xgb_bo_params["max_depth"]= int(xgb_bo_params["max_depth"])
xgb_bo_params["n_estimators"]= int(xgb_bo_params["n_estimators"])
for param in ["n_estimators", "max_depth", "gamma", "learning_rate"]:
    xgb_bo_params["xgb__" + param] = xgb_bo_params.pop(param)

In [17]:
xgb_bo_params

{'xgb__n_estimators': 59,
 'xgb__max_depth': 3,
 'xgb__gamma': 0.30965886088820815,
 'xgb__learning_rate': 0.5060008606348984}

## Comparing the best sets of parameters

In [18]:
xgb_pipe = xgb_pipe.set_params(**xgb_gs_params)
xgb_gs_cv_rmse = cross_val_score(xgb_pipe, X_train, y_train, cv = 5, scoring = "neg_root_mean_squared_error")
print("Median RMSE for XGB with grid search: {}".format(round(np.median(xgb_gs_cv_rmse) * -1, 4)))

Median RMSE for XGB with grid search: 445160.3388


In [19]:
xgb_pipe = xgb_pipe.set_params(**xgb_bo_params)
xgb_bo_cv_rmse = cross_val_score(xgb_pipe, X_train, y_train, cv = 5, scoring = "neg_root_mean_squared_error")
print("Median RMSE for XGB with Bayesian optimization: {}".format(round(np.median(xgb_bo_cv_rmse) * -1, 4)))

Median RMSE for XGB with Bayesian optimization: 458227.1717


In [20]:
if np.median(xgb_gs_cv_rmse) > np.median(xgb_bo_cv_rmse):
    xgb_params = xgb_gs_params
else:
    xgb_params = xgb_bo_params
xgb_model = xgb_pipe.set_params(**xgb_params)
res = xgb_model.fit(X_train, y_train)

In [21]:
xgb_pred = xgb_model.predict(X_test)
print("XGBoost testing results", 
      "\nRMSE: {}".format(round(mean_squared_error(y_test, xgb_pred, squared = False), 4)), 
      "\nMAE: {}".format(round(mean_absolute_error(y_test, xgb_pred), 4)), 
      "\nMAPE: {}".format(round(mean_absolute_percentage_error(y_test, xgb_pred), 4)))

XGBoost testing results 
RMSE: 416816.2556 
MAE: 283611.8762 
MAPE: 0.194


# Neural Network

In [22]:
def nn_architecture(hidden_neurons = 64, kernel_initializer = "he_normal", dropout = 0.2, optimizer = "SGD"):
    nn = Sequential()
    nn.add(Dense(hidden_neurons, activation = 'sigmoid', kernel_initializer = kernel_initializer))
    nn.add(Dropout(dropout))
    nn.add(Dense(1, activation = 'relu'))
    nn.compile(optimizer = optimizer, loss = 'MeanSquaredError', metrics = ['MeanSquaredError'])
    return nn

In [23]:
nn = KerasRegressor(build_fn = nn_architecture, verbose = 0)
nn._estimator_type = "regressor"

nn_pipe = Pipeline(
    steps = [("preprocessing", preprocessor), ("nn", nn)]
)

early_stopping = tf.keras.callbacks.EarlyStopping(monitor = 'loss', patience = 10)
nn_param_grid = {
    "nn__hidden_neurons": [32, 64, 96, 128, 192, 256],
    "nn__kernel_initializer": ["uniform"],
    "nn__dropout": [0, 0.1, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45],
    "nn__optimizer": ["SGD"]
}

nn_fixed_params = {
    "nn__callbacks": [early_stopping],
    "nn__epochs": 200,
    "nn__batch_size": 128
}

nn_pipe = nn_pipe.set_params(**nn_fixed_params)
nn_model = GridSearchCV(nn_pipe, nn_param_grid, cv = 5, n_jobs = -1)
res = nn_model.fit(X_train, y_train)

C:\Users\mspya\AppData\Local\Temp/ipykernel_26508/2311668703.py:1: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  nn = KerasRegressor(build_fn = nn_architecture, verbose = 0)


In [24]:
nn_model.best_params_

{'nn__dropout': 0.35,
 'nn__hidden_neurons': 256,
 'nn__kernel_initializer': 'uniform',
 'nn__optimizer': 'SGD'}

In [25]:
nn_pred = nn_model.predict(X_test)
print("Neural Network testing results", 
      "\nRMSE: {}".format(round(mean_squared_error(y_test, nn_pred, squared = False), 4)), 
      "\nMAE: {}".format(round(mean_absolute_error(y_test, nn_pred), 4)), 
      "\nMAPE: {}".format(round(mean_absolute_percentage_error(y_test, nn_pred), 4)))

Neural Network testing results 
RMSE: 452360.9153 
MAE: 327383.7717 
MAPE: 0.2324


# Stacking the models using Ridge regression

In [26]:
rf_pipe.set_params(**random_forest_model.best_params_)
nn_pipe.set_params(**nn_model.best_params_)

estimators = [
    ('en', elastic_net_model),
    ('rf', rf_pipe),
    ('xgb', xgb_model),
    ('nn', nn_pipe)
]

stack_model = StackingRegressor(
    estimators = estimators,
    final_estimator = RidgeCV()
)

res = stack_model.fit(X_train, y_train)

In [27]:
stack_pred = stack_model.predict(X_test)
print("Stacked model testing results", 
      "\nRMSE: {}".format(round(mean_squared_error(y_test, stack_pred, squared = False), 4)), 
      "\nMAE: {}".format(round(mean_absolute_error(y_test, stack_pred), 4)), 
      "\nMAPE: {}".format(round(mean_absolute_percentage_error(y_test, stack_pred), 4)))

Stacked model testing results 
RMSE: 381012.7937 
MAE: 264502.4703 
MAPE: 0.1765


# Results summary

In [28]:
def populate_scores_list(score, preds, y_true, score_params = {}):
    output = []
    for pred in preds:
        output.append(score(y_true, pred, **score_params))
    return output

In [29]:
all_predictions = [elastic_net_pred, rf_pred, xgb_pred, nn_pred, stack_pred]
models = ["Elastic Net", "Random Forest", "XGBoost", "Neural Network", "Stacked Regressor"]
RMSEs = populate_scores_list(mean_squared_error, all_predictions, y_test, {"squared": False})
MAEs = populate_scores_list(mean_absolute_error, all_predictions, y_test)
MAPEs = populate_scores_list(mean_absolute_percentage_error, all_predictions, y_test)
testing_summary = pd.DataFrame({"RMSE": RMSEs, "MAE": MAEs, "MAPE": MAPEs}, index = models)
pd.set_option("display.precision", 3)
print("Models' performance on holdout testing sample")
display(testing_summary.round(3).style.highlight_min(color = "yellow"))

Models' performance on holdout testing sample


,RMSE,MAE,MAPE
Elastic Net,392897.423,282319.037,0.198
Random Forest,442219.140,301930.278,0.205
XGBoost,416816.256,283611.876,0.194
Neural Network,452360.915,327383.772,0.232
Stacked Regressor,381012.794,264502.470,0.176


**Stalking the models together provides results superior to any individual model**

# Saving the final model

In [30]:
import pickle
with open("sf_sold_houses_model.pkl", "wb") as f:
    pickle.dump(stack_model, f)

INFO:tensorflow:Assets written to: ram://384bc523-0eee-4a73-b84a-cd3126e0a8e3/assets
